In [1]:
import socket
from TikTokLive import TikTokLiveClient
from TikTokLive.events import CommentEvent, GiftEvent, JoinEvent
# Configuração UDP para o ESP8266
UDP_IP = "192.168.1.XX"  # Coloque o IP do seu ESP aqui
UDP_PORT = 4210
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

client = TikTokLiveClient(unique_id="@wagnermaker")

def send_to_esp(s1, s2, led):
    msg = f"{s1},{s2},{led}"
    sock.sendto(msg.encode(), (UDP_IP, UDP_PORT))

# @client.on(JoinEvent)
# async def on_join(event: JoinEvent):
#     # O unique_id é o @ do usuário, nickname é o nome que aparece na tela
#     nome = event.user.nickname
#     user_at = event.user.unique_id
    
#     print(f"Seja bem vindo {nome} (@{user_at}) acabou de entrar na live!")
    
#     # Dica de Mecatrônico: Fazer o robô dar uma "balançadinha" de boas-vindas
#     # Servo 1 vai para 80 graus, depois volta para 90 (exemplo)
#     send_to_esp(100, 90, 1)
    
@client.on(CommentEvent)

async def on_comment(event: CommentEvent):
    print(f"{event.user.unique_id} disse: {event.comment}")
    # Exemplo: Comentários movem o robô de leve
    if "salve" in event.comment.lower():
        # send_to_esp(45, 90, 1)
        print(f" {event.user.nickname} (@{event.user.unique_id}) Acabou de mandar um salve!!!")
    elif "direita" in event.comment.lower():
        send_to_esp(135, 90, 1)


@client.on(GiftEvent)

async def on_gift(event: GiftEvent):
    # Isso gera muito engajamento: Reagir a presentes
    print(f"Recebemos um {event.gift.name} de {event.user.nickname}!")
    # Movimento especial de "agradecimento" ou "loucura"
    send_to_esp(90, 180, 1) # Ex: Servo 2 sobe tudo
    # Aqui você poderia chamar uma API de IA para gerar uma frase de agradecimento

await client.start()

CancelledError: 

In [ ]:
import serial
import tkinter as tk
from queue import Queue
import math
import asyncio
import time
import random
from threading import Thread
from TikTokLive import TikTokLiveClient
from TikTokLive.events import CommentEvent, GiftEvent, JoinEvent

# --- CONFIGURAÇÃO SERIAL ---
PORTA_COM = 'COM7' 
try:
    ser = serial.Serial(PORTA_COM, 115200, timeout=0.1)
    time.sleep(2)
    print(f"✅ [SISTEMA] Arduino conectado em {PORTA_COM}")
except:
    print("❌ [ERRO] Arduino não encontrado.")
    ser = None

msg_queue = Queue()

def send_to_arduino(s1, s2, led):
    if ser:
        comando = f"{s1},{s2},{led}\n"
        ser.write(comando.encode())

# --- DANÇAS ELABORADAS (INTELIGÊNCIA NO PYTHON) ---

async def danca_rosa():
    print("🌹 [DANÇA] Executando coreografia da Rosa...")
    for _ in range(20):
        send_to_arduino(random.randint(0,180), random.randint(0,180), 3)
        await asyncio.sleep(0.15)
    send_to_arduino(30, 90, 0)
    time.sleep(0.5)
    send_to_arduino(120, 60, 1)
    time.sleep(0.5)
    send_to_arduino(0, 90, 0)
    time.sleep(0.5)
    send_to_arduino(60, 120, 1)
    time.sleep(0.5)
    send_to_arduino(96, 90, 0)

async def danca_infinito():
    print("♾️ [DANÇA] Executando movimento Infinito...")
    
    # Parte 1: Balanço em oposição (Suave e Artístico)
    # Dura mais tempo e cobre ângulos maiores
    for i in range(120):  # Aproximadamente 6-8 segundos
        # S1 e S2 se movem em espelho usando seno e cosseno
        angulo_s1 = 90 + 80 * math.sin(i * 0.1)
        angulo_s2 = 90 + 80 * math.sin(i * 0.1 + math.pi) # Invertido 180°
        
        # Alterna LEDs entre 1 e 2 suavemente conforme o ângulo
        led = 1 if angulo_s1 > 90 else 2
        
        send_to_arduino(angulo_s1, angulo_s2, led)
        await asyncio.sleep(0.05)

    # Parte 2: O "Grand Finale" (Crescendo)
    # Movimento rápido de fechamento com LED 3 (alerta/comemoração)
    for _ in range(3):
        send_to_arduino(0, 180, 3)
        await asyncio.sleep(0.3)
        send_to_arduino(180, 0, 3)
        await asyncio.sleep(0.3)

    # Reset suave para a posição central
    send_to_arduino(90, 90, 0)

async def danca_coracao():
    print("❤️ [DANÇA] Executando batida do Coração...")
    for _ in range(3):
        send_to_arduino(110, 70, 2) # Contração
        await asyncio.sleep(0.1)
        send_to_arduino(70, 110, 2) # Expansão
        await asyncio.sleep(0.4)
    send_to_arduino(90, 90, 0)

async def danca_especial():
    print("🎉 [DANÇA] Comemoração Aleatória!")
    for _ in range(20):
        send_to_arduino(random.randint(0,180), random.randint(0,180), 3)
        await asyncio.sleep(0.15)
    send_to_arduino(30, 90, 0)
    time.sleep(0.5)
    send_to_arduino(120, 60, 1)
    time.sleep(0.5)
    send_to_arduino(0, 90, 0)
    time.sleep(0.5)
    send_to_arduino(60, 120, 1)
    time.sleep(0.5)
    send_to_arduino(96, 90, 0)

# --- EVENTOS TIKTOK ---
client = TikTokLiveClient(unique_id="@wagnermaker")

# @client.on(JoinEvent)
# async def on_join(event: JoinEvent):
#     print(f"👤 [JOIN] {event.user.nickname}")
#     msg_queue.put((f"BEM-VINDO\n{event.user.nickname}", "white"))
#     send_to_arduino(120, 60, 1)
#     await asyncio.sleep(1)
#     send_to_arduino(90, 90, 0)

@client.on(CommentEvent)
async def on_comment(event: CommentEvent):
    print(f"💬 [CHAT] {event.user.unique_id}: {event.comment}")
    msg_queue.put((f"{event.user.nickname}\ndisse: {event.comment}", "#00FFFF"))
    send_to_arduino(0, 10, 1) # Só pisca o LED azul no comando
    time.sleep(1)
    send_to_arduino(90, 90, 0)
    time.sleep(1)
    send_to_arduino(10, 0, 1)
    time.sleep(1)
    send_to_arduino(90, 90, 0)
    if event.comment.lower().find("dança") != -1:
        await danca_especial()
    if event.comment.lower().find("rosa") != -1:
        await danca_rosa()
    if event.comment.lower().find("coração") != -1 or event.comment.lower().find("coracao") != -1:
        await danca_coracao()
        
        

@client.on(GiftEvent)
async def on_gift(event: GiftEvent):
    nome = event.gift.name.lower()
    msg_queue.put((f"PRESENTE!\n{event.user.nickname} enviou {event.gift.name}", "gold"))
    print(f"🎁 [GIFT] Recebido: {event.gift.name}")

    if "rose" in nome or "rosa" in nome:
        await danca_rosa()
    elif "heart" in nome or "coração" in nome:
        await danca_coracao()
    else:
        await danca_especial()

# --- INTERFACE E THREADS ---
root = tk.Tk()
root.title("LIVE HUB")
root.geometry("600x400")
root.configure(bg='blue')
#Titulo da janela
lavel_titulo = tk.Label(root, text="Arduino integrado \nao chat do tiktok \nMande um presente\n e veja a magica acontecer \n", font=("Arial", 32, "bold"), fg="white", bg="green")
lavel_titulo.pack()
label_nome = tk.Label(root, text="Aguardando...", font=("Arial", 28, "bold"), fg="yellow", bg="green", wraplength=500)
label_nome.pack(expand=True)

def process_queue():
    while not msg_queue.empty():
        texto, cor = msg_queue.get_nowait()
        label_nome.config(text=texto, fg=cor)
    root.after(100, process_queue)

def start_tiktok():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    client.run()

Thread(target=start_tiktok, daemon=True).start()
process_queue()
root.mainloop()

✅ [SISTEMA] Arduino conectado em COM7
💬 [CHAT] mauricio_drzx: que quando mandamos a mensagem
💬 [CHAT] wagnermaker: teste
💬 [CHAT] mauricio_drzx: ele não esquece
💬 [CHAT] lgzinbw: quanto tempo de carreira??
💬 [CHAT] mauricio_drzx: a mensagem
💬 [CHAT] mauricio_drzx: geralmente ele era para ele esquecer a mensagem anterior e esmaecer, mas ele não tá fazendo isso
💬 [CHAT] mauricio_drzx: que quando mandamos a mensagem
💬 [CHAT] wagnermaker: teste
💬 [CHAT] mauricio_drzx: ele não esquece
💬 [CHAT] lgzinbw: quanto tempo de carreira??
💬 [CHAT] mauricio_drzx: a mensagem
💬 [CHAT] mauricio_drzx: geralmente ele era para ele esquecer a mensagem anterior e esmaecer, mas ele não tá fazendo isso
💬 [CHAT] wagnermaker: teste
💬 [CHAT] wagnermaker: teste
💬 [CHAT] wagnermaker: teste


💬 [CHAT] wagnermaker: travou tudo aqui


: 